In [3]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
%matplotlib inline
pd.options.mode.chained_assignment = None  # default='warn'
path = '/Users/ahelgeso/Documents/GitHub/bball-master/Stat Sheets'
os.chdir(path)

### Contains formulas for:
    1. Aligning opponent stats
    2. Gathering n-game rolling averages
    3. Gathering season cumulative averages
    4. Aligning opponent rolling averages
    5. Gathering season rolling average deltas over opponents
    6. Building a cumulative league averages database
    7. Aligning opponent rolling average deltas
    8. American to decimal odds
    9. What the odds should be based on the line
    10. Building running visual record
    11. Building running win pct
    12. Aligning opponent win pct
    13. Calculating rest days between games
    14. Aligning opponent rest days
    15. Calculating implied probability from ML odds

In [4]:
db = pd.read_csv('2015stats.csv')

In [8]:
names = ['2015stats','2016stats','2017stats','2018stats','2019stats']
bothnames = [('2015stats.csv','2015LeagueStats.csv'),('2016stats.csv','2016LeagueStats.csv'),
             ('2017stats.csv','2017LeagueStats.csv'),('2018stats.csv','2018LeagueStats.csv'),
             ('2019stats.csv', '2019LeagueStats.csv')]

In [43]:
def opp_stats(db):
    
    db['Opp EFG'] = np.zeros(len(db['Team']))
    db['Opp TOV'] = np.zeros(len(db['Team']))
    db['Opp ORB'] = np.zeros(len(db['Team']))
    db['Opp FTR'] = np.zeros(len(db['Team']))
    db['Opp Pace'] = np.zeros(len(db['Team']))
    db['Opp OEff'] = np.zeros(len(db['Team']))
    db['Opp DEff'] = np.zeros(len(db['Team']))

    for i, team in enumerate(db['Team']):
        mask = (db['Date'] == db['Date'][i]) & (db['Opponent'] == team)
        db['Opp EFG'][i] = db[mask]['EFG%']
        db['Opp TOV'][i] = db[mask]['TOV%']
        db['Opp ORB'][i] = db[mask]['ORB%']
        db['Opp FTR'][i] = db[mask]['FTR']
        db['Opp Pace'][i] = db[mask]['Pace']
        db['Opp OEff'][i] = db[mask]['OEff']
        db['Opp DEff'][i] = db[mask]['DEff']
        
    return db

In [44]:
### X game rolling averages

def rolling_averages(num, db):
    name = 'Rolling{}EFG'.format(num)
    EFGDB = pd.DataFrame()
    rollingEFG = []
    for team in db['Team'].unique():
        teamRollingEFG = db[db['Team'] == team]['EFG%'].rolling(num).mean()
        rollingEFG.append(teamRollingEFG)
        EFG = {name:teamRollingEFG}
        EFGdf = pd.DataFrame.from_dict(EFG)
        EFGdf = EFGdf.shift(1)
        EFGDB = pd.concat([EFGDB, EFGdf])
    
    name = 'Rolling{}TOV'.format(num)
    TOVDB = pd.DataFrame()
    rollingTOV = []
    for team in db['Team'].unique():
        teamRollingTOV = db[db['Team'] == team]['TOV%'].rolling(num).mean()
        rollingTOV.append(teamRollingTOV)
        TOV = {name:teamRollingTOV}
        TOVdf = pd.DataFrame.from_dict(TOV)
        TOVdf = TOVdf.shift(1)
        TOVDB = pd.concat([TOVDB, TOVdf])

    name = 'Rolling{}ORB'.format(num)
    ORBDB = pd.DataFrame()
    rollingORB = []
    for team in db['Team'].unique():
        teamRollingORB = db[db['Team'] == team]['ORB%'].rolling(num).mean()
        rollingORB.append(teamRollingORB)
        ORB = {name:teamRollingORB}
        ORBdf = pd.DataFrame.from_dict(ORB)
        ORBdf = ORBdf.shift(1)
        ORBDB = pd.concat([ORBDB, ORBdf])

    name = 'Rolling{}FTR'.format(num)
    FTRDB = pd.DataFrame()
    rollingFTR = []
    for team in db['Team'].unique():
        teamRollingFTR = db[db['Team'] == team]['FTR'].rolling(num).mean()
        rollingFTR.append(teamRollingFTR)
        FTR = {name:teamRollingFTR}
        FTRdf = pd.DataFrame.from_dict(FTR)
        FTRdf = FTRdf.shift(1)
        FTRDB = pd.concat([FTRDB, FTRdf])

    name = 'Rolling{}Pace'.format(num)
    PaceDB = pd.DataFrame()
    rollingPoss = []
    for team in db['Team'].unique():
        teamRollingPoss = db[db['Team'] == team]['Pace'].rolling(num).mean()
        rollingPoss.append(teamRollingPoss)
        Pace = {name:teamRollingPoss}
        PACEdf = pd.DataFrame.from_dict(Pace)
        PACEdf = PACEdf.shift(1)
        PaceDB = pd.concat([PaceDB, PACEdf])

    name = 'Rolling{}OEff'.format(num)
    OEffDB = pd.DataFrame()
    rollingOEff = []
    for team in db['Team'].unique():
        teamRollingOEff = db[db['Team'] == team]['OEff'].rolling(num).mean()
        rollingOEff.append(teamRollingOEff)
        OEff = {name:teamRollingOEff}
        OEffdf = pd.DataFrame.from_dict(OEff)
        OEffdf = OEffdf.shift(1)
        OEffDB = pd.concat([OEffDB, OEffdf])

    name = 'Rolling{}DEff'.format(num)
    DEffDB = pd.DataFrame()
    rollingDEff = []
    for team in db['Team'].unique():
        teamRollingDEff = db[db['Team'] == team]['DEff'].rolling(num).mean()
        rollingDEff.append(teamRollingDEff)
        DEff = {name:teamRollingDEff}
        DEffdf = pd.DataFrame.from_dict(DEff)
        DEffdf = DEffdf.shift(1)
        DEffDB = pd.concat([DEffDB, DEffdf])

    db = pd.concat([db, EFGDB, TOVDB, ORBDB, FTRDB, PaceDB, OEffDB, DEffDB], axis =1)
    return db

In [33]:
### season long cumulative averages

def cum_avgs(db):
    name = 'SeasonEFG'
    EFGDB = pd.DataFrame()
    for team in db['Team'].unique():
        rollingEFG = []
        for i in range(len(db[db['Team']==team])):
            teamRollingEFG = db[db['Team'] == team]['EFG%'][:i+1].mean()
            rollingEFG.append(teamRollingEFG)
        EFG = {name:rollingEFG}
        EFGdf = pd.DataFrame.from_dict(EFG)
        EFGdf = EFGdf.shift(1)
        EFGDB = pd.concat([EFGDB, EFGdf])
        EFGDB.reset_index(drop = True, inplace = True)
    
    name = 'SeasonTOV'
    TOVDB = pd.DataFrame()
    for team in db['Team'].unique():
        rollingTOV = []
        for i in range(len(db[db['Team']==team])):
            teamRollingTOV = db[db['Team'] == team]['TOV%'][:i+1].mean()
            rollingTOV.append(teamRollingTOV)
        TOV = {name:rollingTOV}
        TOVdf = pd.DataFrame.from_dict(TOV)
        TOVdf = TOVdf.shift(1)
        TOVDB = pd.concat([TOVDB, TOVdf])
        TOVDB.reset_index(drop = True, inplace = True)

    name = 'SeasonORB'
    ORBDB = pd.DataFrame()
    for team in db['Team'].unique():
        rollingORB = []
        for i in range(len(db[db['Team']==team])):
            teamRollingORB = db[db['Team'] == team]['ORB%'][:i+1].mean()
            rollingORB.append(teamRollingORB)
        ORB = {name:rollingORB}
        ORBdf = pd.DataFrame.from_dict(ORB)
        ORBdf = ORBdf.shift(1)
        ORBDB = pd.concat([ORBDB, ORBdf])
        ORBDB.reset_index(drop = True, inplace = True)

    name = 'SeasonFTR'
    FTRDB = pd.DataFrame()
    for team in db['Team'].unique():
        rollingFTR = []
        for i in range(len(db[db['Team']==team])):
            teamRollingFTR = db[db['Team'] == team]['FTR'][:i+1].mean()
            rollingFTR.append(teamRollingFTR)
        FTR = {name:rollingFTR}
        FTRdf = pd.DataFrame.from_dict(FTR)
        FTRdf = FTRdf.shift(1)
        FTRDB = pd.concat([FTRDB, FTRdf])
        FTRDB.reset_index(drop = True, inplace = True)

    name = 'SeasonPace'
    PaceDB = pd.DataFrame()
    for team in db['Team'].unique():
        rollingPoss = []
        for i in range(len(db[db['Team']==team])):
            teamRollingPoss = db[db['Team'] == team]['Pace'][:i+1].mean()
            rollingPoss.append(teamRollingPoss)
        Pace = {name:rollingPoss}
        PACEdf = pd.DataFrame.from_dict(Pace)
        PACEdf = PACEdf.shift(1)
        PaceDB = pd.concat([PaceDB, PACEdf])
        PaceDB.reset_index(drop = True, inplace = True)

    name = 'SeasonOEff'
    OEffDB = pd.DataFrame()
    for team in db['Team'].unique():
        rollingOEff = []
        for i in range(len(db[db['Team']==team])):  
            teamRollingOEff = db[db['Team'] == team]['OEff'][:i+1].mean()
            rollingOEff.append(teamRollingOEff)
        OEff = {name:rollingOEff}
        OEffdf = pd.DataFrame.from_dict(OEff)
        OEffdf = OEffdf.shift(1)
        OEffDB = pd.concat([OEffDB, OEffdf])
        OEffDB.reset_index(drop = True, inplace = True)

    name = 'SeasonDEff'
    DEffDB = pd.DataFrame()
    for team in db['Team'].unique():
        rollingDEff = []
        for i in range(len(db[db['Team']==team])):  
            teamRollingDEff = db[db['Team'] == team]['DEff'][:i+1].mean()
            rollingDEff.append(teamRollingDEff)
        DEff = {name:rollingDEff}
        DEffdf = pd.DataFrame.from_dict(DEff)
        DEffdf = DEffdf.shift(1)
        DEffDB = pd.concat([DEffDB, DEffdf])
        DEffDB.reset_index(drop = True, inplace = True)
    
#     for x in [db, EFGDB, TOVDB, ORBDB, FTRDB, PaceDB, OEffDB, DEffDB]:
#         print(np.shape(x))
        
    db = pd.concat([db, EFGDB, TOVDB, ORBDB, FTRDB, PaceDB, OEffDB, DEffDB], axis =1)
    return db

In [17]:
def opp_rolling_averages(num, db):

    db['Opp R{}EFG'.format(num)] = np.zeros(len(db['Team']))
    db['Opp R{}TOV'.format(num)] = np.zeros(len(db['Team']))
    db['Opp R{}ORB'.format(num)] = np.zeros(len(db['Team']))
    db['Opp R{}FTR'.format(num)] = np.zeros(len(db['Team']))
    db['Opp R{}Pace'.format(num)] = np.zeros(len(db['Team']))
    db['Opp R{} OEff'.format(num)] = np.zeros(len(db['Team']))
    db['Opp R{} DEff'.format(num)] = np.zeros(len(db['Team']))
    
    for i, opp in enumerate(db['Opponent']):

        db['Opp R{}EFG'.format(num)][i] = db[(db['Date'] == db['Date'][i]) & (db['Team'] == opp)]['Rolling{}EFG'.format(num)]
        db['Opp R{}TOV'.format(num)][i] = db[(db['Date'] == db['Date'][i]) & (db['Team'] == opp)]['Rolling{}TOV'.format(num)]
        db['Opp R{}ORB'.format(num)][i] = db[(db['Date'] == db['Date'][i]) & (db['Team'] == opp)]['Rolling{}ORB'.format(num)]
        db['Opp R{}FTR'.format(num)][i] = db[(db['Date'] == db['Date'][i]) & (db['Team'] == opp)]['Rolling{}FTR'.format(num)]
        db['Opp R{}Pace'.format(num)][i] = db[(db['Date'] == db['Date'][i]) & (db['Team'] == opp)]['Rolling{}Pace'.format(num)]
        db['Opp R{} OEff'.format(num)][i] = db[(db['Date'] == db['Date'][i]) & (db['Team'] == opp)]['Rolling{}OEff'.format(num)]
        db['Opp R{} DEff'.format(num)][i] = db[(db['Date'] == db['Date'][i]) & (db['Team'] == opp)]['Rolling{}DEff'.format(num)]
        
    return db

In [18]:
### X game rolling deltas OVER OPPONENTS

def rolling_delta(num, db):
    name = 'Rolling{}delEFG'.format(num)
    EFGDB = pd.DataFrame()
    rollingEFG = []
    for team in db['Team'].unique():
        teamRollingEFG = db[db['Team'] == team]['delEFG'].rolling(num).mean()
        rollingEFG.append(teamRollingEFG)
        EFG = {name:teamRollingEFG}
        EFGdf = pd.DataFrame.from_dict(EFG)
        EFGdf = EFGdf.shift(1)
        EFGDB = pd.concat([EFGDB, EFGdf])
    
    name = 'Rolling{}delTOV'.format(num)
    TOVDB = pd.DataFrame()
    rollingTOV = []
    for team in db['Team'].unique():
        teamRollingTOV = db[db['Team'] == team]['delTOV'].rolling(num).mean()
        rollingTOV.append(teamRollingTOV)
        TOV = {name:teamRollingTOV}
        TOVdf = pd.DataFrame.from_dict(TOV)
        TOVdf = TOVdf.shift(1)
        TOVDB = pd.concat([TOVDB, TOVdf])

    name = 'Rolling{}delORB'.format(num)
    ORBDB = pd.DataFrame()
    rollingORB = []
    for team in db['Team'].unique():
        teamRollingORB = db[db['Team'] == team]['delORB'].rolling(num).mean()
        rollingORB.append(teamRollingORB)
        ORB = {name:teamRollingORB}
        ORBdf = pd.DataFrame.from_dict(ORB)
        ORBdf = ORBdf.shift(1)
        ORBDB = pd.concat([ORBDB, ORBdf])

    name = 'Rolling{}delFTR'.format(num)
    FTRDB = pd.DataFrame()
    rollingFTR = []
    for team in db['Team'].unique():
        teamRollingFTR = db[db['Team'] == team]['delFTR'].rolling(num).mean()
        rollingFTR.append(teamRollingFTR)
        FTR = {name:teamRollingFTR}
        FTRdf = pd.DataFrame.from_dict(FTR)
        FTRdf = FTRdf.shift(1)
        FTRDB = pd.concat([FTRDB, FTRdf])

    name = 'Rolling{}delEff'.format(num)
    OEffDB = pd.DataFrame()
    rollingOEff = []
    for team in db['Team'].unique():
        teamRollingOEff = db[db['Team'] == team]['delEff'].rolling(num).mean()
        rollingOEff.append(teamRollingOEff)
        OEff = {name:teamRollingOEff}
        OEffdf = pd.DataFrame.from_dict(OEff)
        OEffdf = OEffdf.shift(1)
        OEffDB = pd.concat([OEffDB, OEffdf])

    db = pd.concat([db, EFGDB, TOVDB, ORBDB, FTRDB, OEffDB], axis =1)
    return db

In [19]:
def league_avgs(db):
    
    leagueavg = pd.DataFrame(columns = ['Date', 'LeagueORB','LeagueEFG','LeagueTOV','LeagueFTR','LeagueEff'])
    
    db = db.sort_values(by = 'Date').reset_index(drop = True)
    for date in db['Date'].unique():
        dayORB = db[db['Date']==date]['ORB%'].mean()

        dayEFG = db[db['Date']==date]['EFG%'].mean()

        dayTOV = db[db['Date']==date]['TOV%'].mean()

        dayFTR = db[db['Date']==date]['FTR'].mean()

        dayEff = db[db['Date']==date]['OEff'].mean()

        pd1 = pd.DataFrame([[date, dayORB, dayEFG, dayTOV, dayFTR, dayEff]], 
                                      columns = ['Date', 'LeagueORB','LeagueEFG','LeagueTOV','LeagueFTR','LeagueEff'])

        leagueavg = pd.concat([leagueavg, pd1], ignore_index= True)
        
    cols = ['LeagueORB','LeagueEFG','LeagueTOV','LeagueFTR','LeagueEff']
    LeagueAvgs = pd.DataFrame()
    
    for col in cols:

        LeagueCumAvg = []

        for i in range(len(leagueavg[col])):
            new = leagueavg[col][:i+1].mean()
            LeagueCumAvg.append(new)

        LeagueAvgs = pd.concat([LeagueAvgs, pd.DataFrame({col:LeagueCumAvg}).shift(1)], axis = 1)
    
    LeagueAvgs = pd.concat([leagueavg['Date'], LeagueAvgs], axis = 1)
    return LeagueAvgs

In [20]:
### Align opponent x game rolling deltas

def opp_rolling_dels(num, db):

    db['Opp R{}delEFG'.format(num)] = np.zeros(len(db['Team']))
    db['Opp R{}delTOV'.format(num)] = np.zeros(len(db['Team']))
    db['Opp R{}delORB'.format(num)] = np.zeros(len(db['Team']))
    db['Opp R{}delFTR'.format(num)] = np.zeros(len(db['Team']))
    db['Opp R{} delEff'.format(num)] = np.zeros(len(db['Team']))
    
    for i, opp in enumerate(db['Opponent']):

        db['Opp R{}delEFG'.format(num)][i] = db[(db['Date'] == db['Date'][i]) & (db['Team'] == opp)]['Rolling{}delEFG'.format(num)]
        db['Opp R{}delTOV'.format(num)][i] = db[(db['Date'] == db['Date'][i]) & (db['Team'] == opp)]['Rolling{}delTOV'.format(num)]
        db['Opp R{}delORB'.format(num)][i] = db[(db['Date'] == db['Date'][i]) & (db['Team'] == opp)]['Rolling{}delORB'.format(num)]
        db['Opp R{}delFTR'.format(num)][i] = db[(db['Date'] == db['Date'][i]) & (db['Team'] == opp)]['Rolling{}delFTR'.format(num)]
        db['Opp R{} delEff'.format(num)][i] = db[(db['Date'] == db['Date'][i]) & (db['Team'] == opp)]['Rolling{}delEff'.format(num)]
        
    return db

In [21]:
### Convert american to decimal odds

def am_to_dec_odds(db):
    decOdds = []
    
    for i, odds in enumerate(db['Odds']):
        if odds > 0:
            decOdds.append((odds/100) + 1)
        elif odds < 0:
            decOdds.append((100/abs(odds))+1)
        else:
            decOdds.append(np.nan)
    
    return pd.concat([db, pd.DataFrame({'DecimalOdds':decOdds})], axis = 1)

In [22]:
def pred_odds(db, a=0.675, b=0.183, c=1.165):
    
    predOdds = []
    
    for odds in db['Line']:
        predOdds.append(a * np.exp(b * odds) + c)
    
    return pd.concat([db, pd.DataFrame({'Estimate Odds': predOdds})], axis = 1)

In [23]:
### Build record column

def build_records(db):

    runrec = pd.DataFrame()

    for team in db['Team'].unique():

        rundb = db[db['Team'] == team]

        record = []
        winpct = []
        w = 0
        l = 0

        for res in rundb['Result']:
            if res == 1:
                w+= 1
            else:
                l+= 1
            record.append(f'({w}-{l})')

        runrec = pd.concat([runrec, pd.DataFrame({'Record':record})])

    
    runrec.reset_index(drop = True, inplace = True)
    db = pd.concat([db, runrec], axis = 1)
    return db

In [24]:
### Build record column

def build_win_pct(db):

    runpct = pd.DataFrame()

    for team in db['Team'].unique():

        rundb = db[db['Team'] == team]

        winpct = []
        w = 0
        l = 0

        for res in rundb['Result']:
            if res == 1:
                w+= 1
            else:
                l+= 1
            winpct.append(w/(w+l))

        runpct = pd.concat([runpct, pd.DataFrame({'Win Pct':winpct}).shift(1)])

    runpct.reset_index(drop = True, inplace = True)
    db = pd.concat([db, runpct], axis = 1)
    return db

In [52]:
def opp_win_pct(db):

    db['Opp Win Pct'] = np.zeros(len(db))
    
    for i, opp in enumerate(db['Opponent']):

        db['Opp Win Pct'][i] = db[(db['Date'] == db['Date'][i]) & (db['Team'] == opp)]['Win Pct']
         
    return db

In [ ]:
def rest_days(db):
    
    restLog = pd.DataFrame()

    for i, team in enumerate(list(db['Team'].unique())):

        rundb = db[db['Team'] == team]
        dates = list(pd.to_datetime(rundb['Date'], format = '%Y%m%d'))
        runRest = []

        for j in range(len(dates)):

            if j == 0:
                runRest.append(np.nan)

            else:
                rest = dates[j] - dates[j-1]
                rest = int(rest.days) - 1
                runRest.append(rest)

        restLog = pd.concat([restLog,(pd.DataFrame({'Rest':runRest}))])
        
    return pd.concat([db, restLog.reset_index(drop = True)], axis = 1)

In [64]:
def opp_rest_days(db):

    db['Opp Rest'] = np.zeros(len(db))
    
    for i, opp in enumerate(db['Opponent']):

        db['Opp Rest'][i] = db[(db['Date'] == db['Date'][i]) & (db['Team'] == opp)]['Rest']
         
    return db


In [76]:
def implied_odds(db):
    
    db['Implied Odds'] = np.zeros(len(db))
    
    for i in range(len(db['Implied Odds'])):
        
        db['Implied Odds'][i] = 1/db['DecimalOdds'][i]
    
    return db

In [10]:
def vs_league(db, league_db):
    
    db['Rolling7EFG_delLeague'] = np.zeros(len(db))
    db['Rolling7ORB_delLeague'] = np.zeros(len(db))
    db['Rolling7TOV_delLeague'] = np.zeros(len(db))
    db['Rolling7FTR_delLeague'] = np.zeros(len(db))
    db['Rolling7OEff_delLeague'] = np.zeros(len(db))
    db['Rolling7DEff_delLeague'] = np.zeros(len(db))
    
    db['SeasonEFG_delLeague'] = np.zeros(len(db))
    db['SeasonORB_delLeague'] = np.zeros(len(db))
    db['SeasonTOV_delLeague'] = np.zeros(len(db))
    db['SeasonFTR_delLeague'] = np.zeros(len(db))
    db['SeasonOEff_delLeague'] = np.zeros(len(db))
    db['SeasonDEff_delLeague'] = np.zeros(len(db))
    
    for i, date in enumerate(db['Date']):
        db['Rolling7EFG_delLeague'][i] = (db['Rolling7EFG'][i] - league_db[league_db['Date']==date]['LeagueEFG'])*100.0
        db['Rolling7ORB_delLeague'][i] = (db['Rolling7ORB'][i] - league_db[league_db['Date']==date]['LeagueORB'])*100.0
        db['Rolling7TOV_delLeague'][i] = (db['Rolling7TOV'][i] - league_db[league_db['Date']==date]['LeagueTOV'])*100.0
        db['Rolling7FTR_delLeague'][i] = (db['Rolling7FTR'][i] - league_db[league_db['Date']==date]['LeagueFTR'])*100.0
        db['Rolling7OEff_delLeague'][i] = db['Rolling7OEff'][i] - league_db[league_db['Date']==date]['LeagueEff']
        db['Rolling7DEff_delLeague'][i] = db['Rolling7DEff'][i] - league_db[league_db['Date']==date]['LeagueEff']
    
        db['SeasonEFG_delLeague'][i] = (db['SeasonEFG'][i] - league_db[league_db['Date']==date]['LeagueEFG'])*100.0
        db['SeasonORB_delLeague'][i] = (db['SeasonORB'][i] - league_db[league_db['Date']==date]['LeagueORB'])*100.0
        db['SeasonTOV_delLeague'][i] = (db['SeasonTOV'][i] - league_db[league_db['Date']==date]['LeagueTOV'])*100.0
        db['SeasonFTR_delLeague'][i] = (db['SeasonFTR'][i] - league_db[league_db['Date']==date]['LeagueFTR'])*100.0
        db['SeasonOEff_delLeague'][i] = db['SeasonOEff'][i] - league_db[league_db['Date']==date]['LeagueEff']
        db['SeasonDEff_delLeague'][i] = db['SeasonDEff'][i] - league_db[league_db['Date']==date]['LeagueEff']
        
    return db
        

In [3]:
### Margin of victory / defeat

for name in ['2016stats','2017stats','2018stats','2019stats']:
    db = pd.read_csv(f'{name}.csv')
    db['Margin'] = db['Team Points'] - db['Opp Points']
    db.to_csv(f'{name}.csv', index = False)

In [55]:
### Align opponent stats, calculate variable rolling averages, align opponent rolling averages

rolling = 7

for name in names:
    db = pd.read_csv(f'{name}.csv')
    db = opp_stats(db)
    db = rolling_averages(rolling, db)
    db = opp_rolling_averages(rolling, db)
    db.to_csv(f'{name}.csv', index = False)

In [56]:
### Find in-game percentage deltas (team - opponent)

for name in names:
    db = pd.read_csv(f'{name}.csv')
    db['delEFG'] = (db['EFG%'] - db['Opp EFG']) * 100
    db['delTOV'] = (db['TOV%'] - db['Opp TOV']) * 100
    db['delORB'] = (db['ORB%'] - db['Opp ORB']) * 100
    db['delFTR'] = (db['FTR'] - db['Opp FTR']) * 100
    db['delEff'] = (db['OEff'] - db['DEff'])
    
    db.to_csv(f'{name}.csv', index = False)

In [57]:
### Calculate rolling four factor differentials during each game

rolling = 10

for name in names:
    db = pd.read_csv(f'{name}.csv')
    db = rolling_delta(rolling, db).round(3)
    db.to_csv(f'{name}.csv', index = False)

In [58]:
### Align opponent rolling deltas

rolling = 10

for name in names:
    db = pd.read_csv(f'{name}.csv')
    db = opp_rolling_dels(rolling, db).round(3)
    db.to_csv(f'{name}.csv', index = False)

In [70]:
### Odds conversion

for name in names:
    db = pd.read_csv(f'{name}.csv')
    db = am_to_dec_odds(db)
    db.to_csv(f'{name}.csv', index = False)

In [60]:
### Add 'Season' Column

for name in names:
    db = pd.read_csv(f'{name}.csv')
    db['Season'] = name[0:4]
    db.to_csv(f'{name}.csv', index = False)

In [61]:
### Win Pct

for name in names:
    db = pd.read_csv(f'{name}.csv')
    db = build_win_pct(db)
    db.to_csv(f'{name}.csv', index = False)

In [62]:
### Opp win Pct

for name in names:
    db = pd.read_csv(f'{name}.csv')
    db = opp_win_pct(db)
    db.to_csv(f'{name}.csv', index = False)

In [63]:
### Rest Days

for name in names:
    db = pd.read_csv(f'{name}.csv')
    db = rest_days(db)
    db.to_csv(f'{name}.csv', index = False)

In [65]:
### Opp Rest Days

for name in names:
    db = pd.read_csv(f'{name}.csv')
    db = opp_rest_days(db)
    db.to_csv(f'{name}.csv', index = False)

In [78]:
### Implied odds

for name in names:
    db = pd.read_csv(f'{name}.csv')
    db = implied_odds(db)
    db.to_csv(f'{name}.csv', index = False)

In [90]:
### Building league avg dbs

names = ['2015','2016','2017','2018','2019']
for name in names:
    db = pd.read_csv(f'{name}stats.csv')
    db = league_avgs(db)
    db.to_csv(f'{name}LeagueStats.csv', index = False)

In [43]:
### Team Season Cumulative Averages

for name in names:
    db = pd.read_csv(f'{name}.csv')
    db = cum_avgs(db)
    db.to_csv(f'{name}.csv', index = False)

In [12]:
for name, league in bothnames:
    db = pd.read_csv(name)
    league_db = pd.read_csv(league)
    db = vs_league(db, league_db)
    db.to_csv(name, index = False)

,Team,Location,Game Number,Team Points,Opp Points,Result,Date,Opponent,EFG%,TOV%,...,Rolling7TOV_delLeague,Rolling7FTR_delLeague,Rolling7OEff_delLeague,Rolling7DEff_delLeague,SeasonEFG_delLeague,SeasonORB_delLeague,SeasonTOV_delLeague,SeasonFTR_delLeague,SeasonOEff_delLeague,SeasonDEff_delLeague
0,CLE,0,1,104,116,0,20181017,TOR,0.441,0.135,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CLE,0,2,123,131,0,20181019,MIN,0.527,0.117,...,NaN,NaN,NaN,NaN,-4.718939,5.167929,1.107576,12.894192,-1.456566,10.443434
2,CLE,1,3,111,133,0,20181021,ATL,0.515,0.146,...,NaN,NaN,NaN,NaN,-2.335141,5.083646,0.266101,8.941737,2.712838,12.512838
3,CLE,1,4,86,102,0,20181024,BRK,0.399,0.135,...,NaN,NaN,NaN,NaN,-1.944880,5.549848,1.268049,2.651433,-0.586768,12.879899
4,CLE,0,5,103,110,0,20181025,DET,0.506,0.086,...,NaN,NaN,NaN,NaN,-4.268008,6.120152,1.299966,1.133934,-5.008524,9.316476
5,CLE,1,6,107,119,0,20181027,IND,0.506,0.136,...,NaN,NaN,NaN,NaN,-3.983143,3.399962,0.281433,0.017245,-4.822266,8.097734
6,CLE,1,7,136,114,1,20181030,ATL,0.572,0.107,...,NaN,NaN,NaN,NaN,-3.569910,6.626021,0.326523,0.441783,-2.716265,10.233735
7,CLE,1,8,91,110,0,20181101,DEN,0.440,0.125,...,-0.206740,2.409812,0.333859,8.461859,-2.490840,6.388896,-0.192454,2.395526,0.333573,8.462145
8,CLE,0,9,94,126,0,20181103,CHO,0.494,0.129,...,-0.248584,0.505250,-0.014922,9.513078,-3.378052,6.441882,-0.111084,2.117750,-0.832922,8.992078
9,CLE,0,10,100,102,0,20181105,ORL,0.575,0.174,...,-0.258368,-1.152820,-2.810692,10.346308,-3.205018,4.058448,-0.158368,1.236069,-1.875914,10.535197
